## Scrapping bantennews.co.id

In [17]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
from urllib.request import Request, urlopen

In [18]:
jumlah_index = 10
threads_link = []
links = []
results = []
threads = []

In [19]:
def scrape_links(page_number):
    url=f"https://www.bantennews.co.id/category/politik/pemilu-2024/page/{page_number}/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('div',{"class": "td-block-span6"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [20]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 10 links from page 7
Scraped 10 links from page 9
Scraped 10 links from page 3
Scraped 10 links from page 5
Scraped 10 links from page 2
Scraped 10 links from page 1
Scraped 10 links from page 4
Scraped 10 links from page 8
Scraped 10 links from page 10
Scraped 10 links from page 6
Total Links: 100


In [21]:
def scrape_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1',{"class": "entry-title"})
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            # author_elem = soup.find('div', {"class": "tags-wrapper"})
            # if author_elem:
            #     author_text = author_elem.find('span')
            #     author_text = author_text.text.split(':')[1]
            # else:
            author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('time', {"class": "entry-date updated td-module-date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            category_elements = soup.find('li', {"class": "entry-category"})
            if category_elements:
                category_text = category_elements.find('a')
                category_text= category_text.text.strip()
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "td-post-content td-pb-padding-side"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
                content_text ="Content not found"

            results.append({'title': title_text,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'region':'banten',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [22]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [23]:
df = pd.DataFrame(results)
print("Total Hasil ",len(results))
df.head(10)

Total Hasil  100


,title,author,category,date,content,region,link
0,"Anies Baswedan Diamanahkan jadi Capres, Bukan ...",Author not found,Pemilu 2024,"Senin 28 Agu 2023, 06:07 WIB",SERANG – Presiden Partai Keadilan Sejahtera (P...,banten,https://www.bantennews.co.id/anies-baswedan-di...
1,"Tidak Masuk DCS, Bacaleg Laporkan KPU Lebak ke...",Author not found,Pemilu 2024,"Senin 11 Sep 2023, 23:07 WIB",LEBAK – Badan Pengawas Pemilu (Bawaslu) Kabupa...,banten,https://www.bantennews.co.id/tidak-masuk-dcs-b...
2,"Persiapan Pemilu 2024, Polda Banten Tingkatkan...",Author not found,Pemilu 2024,"Minggu 10 Sep 2023, 16:03 WIB",SERANG – Bidang Hubungan Masyarakat (Bidhumas)...,banten,https://www.bantennews.co.id/persiapan-pemilu-...
3,Anies: Surya Paloh Tak Menolak AHY Jadi Bacawa...,Author not found,Pemilu 2024,"Selasa 5 Sep 2023, 15:04 WIB",JAKARTA – Bakal capres Koalisi Perubahan untuk...,banten,https://www.bantennews.co.id/anies-surya-paloh...
4,KPU Minta Pemkot Naikan Hibah Pilkada Kota Serang,Author not found,Pemilu 2024,"Senin 28 Agu 2023, 23:07 WIB",SERANG – KPU Kota Serang meminta Pemkot Serang...,banten,https://www.bantennews.co.id/kpu-minta-pemkot-...
5,"DCS Sudah Ditetapkan, KPU Beri Kesempatan Masy...",Author not found,Pemilu 2024,"Sabtu 19 Agu 2023, 13:20 WIB",BANTEN – Komisi Pemilihan Umum (KPU) memberika...,banten,https://www.bantennews.co.id/dcs-sudah-ditetap...
6,DPD PKS Sudah Siapkan 4 Kandidat untuk Pilkada...,Author not found,Pemilu 2024,"Sabtu 26 Agu 2023, 03:08 WIB","LEBAK – Menjelang Pilkada 2024 mendatang, Dewa...",banten,https://www.bantennews.co.id/dpd-pks-sudah-sia...
7,"Lima Calegnya Eks Napi Koruptor, NasDem: Kami ...",Author not found,Pemilu 2024,"Senin 28 Agu 2023, 09:12 WIB",JAKARTA – Indonesia Corruption Watch (ICW) men...,banten,https://www.bantennews.co.id/lima-calegnya-eks...
8,Terikat Kerja Sama Politik Dukung Ganjar Capre...,Author not found,Pemilu 2024,"Senin 28 Agu 2023, 01:12 WIB",SERANG – Ketua Badan Pemenangan Pemilu (Bappil...,banten,https://www.bantennews.co.id/terikat-kerja-sam...
9,KPU Pandeglang Terima Bendera Kirab Pemilu,Author not found,Pemilu 2024,"Senin 11 Sep 2023, 22:06 WIB",PANDEGLANG – Komisi Pemilihan Umum (KPU) Kabup...,banten,https://www.bantennews.co.id/kpu-pandeglang-te...


In [24]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'./excel/banten_antranews_{current_datetime}.xlsx'
df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')

Data has been saved to ./excel/banten_antranews_2023-09-26_13-00-12.xlsx
